In [1]:
#from sagemaker.local import LocalSession
import sagemaker
from sagemaker import get_execution_role

#sagemaker_session = sagemaker.Session()
sagemaker_session = sagemaker.LocalSession()
#sagemaker_session.config={'local': {'local_code': True}}


# Get a SageMaker-compatible role used by this Notebook Instance.
#role = get_execution_role()
role = "arn:aws:iam::961558279552:role/ravindra"
#train_input = 's3://1814-tigerassign1/B/housing.csv'
# S3 prefix
bucket = sagemaker_session.default_bucket()
prefix = 'SkLinearLearner-pipeline'

In [2]:

#WORK_DIRECTORY = 'housing_data'
WORK_DIRECTORY = '.'

train_input = sagemaker_session.upload_data(
    path='{}/{}'.format(WORK_DIRECTORY, 'housing.csv'), 
    bucket=bucket,
    key_prefix='{}/{}'.format(prefix, 'train'))

# Create SageMaker Scikit Estimator 


In [3]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"
script_path = './sklearn_pipe.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.m4.xlarge",
    #instance_type = "local",
    sagemaker_session=sagemaker_session)


In [4]:
train_input

's3://sagemaker-ap-south-1-961558279552/SkLinearLearner-pipeline/train/housing.csv'

In [5]:
sklearn_preprocessor.fit({'train': train_input})

Creating network "sagemaker-local" with the default driver
Creating mphqp8xzx4-algo-1-6pp0c ... 
Creating mphqp8xzx4-algo-1-6pp0c ... done
Attaching to mphqp8xzx4-algo-1-6pp0c
mphqp8xzx4-algo-1-6pp0c | 2021-06-15 14:45:26,228 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
mphqp8xzx4-algo-1-6pp0c | 2021-06-15 14:45:26,230 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
mphqp8xzx4-algo-1-6pp0c | 2021-06-15 14:45:26,239 sagemaker_sklearn_container.training INFO     Invoking user training script.
mphqp8xzx4-algo-1-6pp0c | 2021-06-15 14:45:27,262 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
mphqp8xzx4-algo-1-6pp0c | 2021-06-15 14:45:27,272 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
mphqp8xzx4-algo-1-6pp0c | 2021-06-15 14:45:27,282 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
mphqp8xzx4-algo-1-6pp0c | 202

# Batch transform our training data 

In [6]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    #instance_type='local',
    assemble_with = 'Line',
    accept = 'text/csv')

In [7]:

# Preprocess training input
transformer.transform(train_input, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

Attaching to n46q4rccqm-algo-1-ngt4c
n46q4rccqm-algo-1-ngt4c | 2021-06-15 14:46:13,918 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
n46q4rccqm-algo-1-ngt4c | 2021-06-15 14:46:13,921 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
n46q4rccqm-algo-1-ngt4c | 2021-06-15 14:46:13,923 INFO - sagemaker-containers - nginx config: 
n46q4rccqm-algo-1-ngt4c | worker_processes auto;
n46q4rccqm-algo-1-ngt4c | daemon off;
n46q4rccqm-algo-1-ngt4c | pid /tmp/nginx.pid;
n46q4rccqm-algo-1-ngt4c | error_log  /dev/stderr;
n46q4rccqm-algo-1-ngt4c | 
n46q4rccqm-algo-1-ngt4c | worker_rlimit_nofile 4096;
n46q4rccqm-algo-1-ngt4c | 
n46q4rccqm-algo-1-ngt4c | events {
n46q4rccqm-algo-1-ngt4c |   worker_connections 2048;
n46q4rccqm-algo-1-ngt4c | }
n46q4rccqm-algo-1-ngt4c | 
n46q4rccqm-algo-1-ngt4c | http {
n46q4rccqm-algo-1-ngt4c |   include /etc/nginx/mime.types;
n46q4rccqm-algo-1-ngt4c |   default_type application/octet-stream;
n46q4rccqm-algo-1-ngt4

# Serial Inference Pipeline with Scikit preprocessor and Linear Learner


In [8]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inference_model = sklearn_preprocessor.create_model()

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix


scikit_learn_inference_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name=endpoint_name)

Attaching to a8n3g7nu4c-algo-1-878hi
a8n3g7nu4c-algo-1-878hi | 2021-06-15 15:02:59,123 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
a8n3g7nu4c-algo-1-878hi | 2021-06-15 15:02:59,125 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
a8n3g7nu4c-algo-1-878hi | 2021-06-15 15:02:59,126 INFO - sagemaker-containers - nginx config: 
a8n3g7nu4c-algo-1-878hi | worker_processes auto;
a8n3g7nu4c-algo-1-878hi | daemon off;
a8n3g7nu4c-algo-1-878hi | pid /tmp/nginx.pid;
a8n3g7nu4c-algo-1-878hi | error_log  /dev/stderr;
a8n3g7nu4c-algo-1-878hi | 
a8n3g7nu4c-algo-1-878hi | worker_rlimit_nofile 4096;
a8n3g7nu4c-algo-1-878hi | 
a8n3g7nu4c-algo-1-878hi | events {
a8n3g7nu4c-algo-1-878hi |   worker_connections 2048;
a8n3g7nu4c-algo-1-878hi | }
a8n3g7nu4c-algo-1-878hi | 
a8n3g7nu4c-algo-1-878hi | http {
a8n3g7nu4c-algo-1-878hi |   include /etc/nginx/mime.types;
a8n3g7nu4c-algo-1-878hi |   default_type application/octet-stream;
a8n3g7nu4c-algo-1-878h

# Make a request to our pipeline endpoint


In [9]:

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

payload = '-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY,452600.0'
actual_rings = 10
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=CSVSerializer())

print(predictor.predict(payload))

a8n3g7nu4c-algo-1-878hi | 2021-06-15 15:03:03,264 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
a8n3g7nu4c-algo-1-878hi | /miniconda3/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:440: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
a8n3g7nu4c-algo-1-878hi |   FutureWarning)
b'{"instances": [{"features": [452600.0, -1.3278352216308462, 1.0525482830366848, 0.9821426581785077, -0.8048190966246049, -0.9724764790070289, -0.9744285971768408, -0.9770328537634586, 2.3447657583017163, 0.0, 0.0, 0.0, 1.0, 0.0]}]}'
a8n3g7nu4c-algo-1-878hi | 172.18.0.1 - - [15/Jun/2021:15:03:03 +0000] "POST /invocations HTTP/1.1" 200 231 "-" "python-urllib3/1.26.4"


# Delete Endpoint

In [10]:

sm_client = sagemaker_session.boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:ap-south-1:961558279552:endpoint/inference-pipeline-ep-2021-06-15-15-02-56".